In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import os
from sklearn.decomposition import PCA
from sklearn import preprocessing


In [2]:
#build table: item vs feature
import pandas as pd

column_names = ["item_ID", "sweetness", "category",'cold_only']#price, calori

item_char = pd.DataFrame(columns = column_names)

initial_data=[[0,'0.9','shake',True],
              [1,'0.3','shake',False],
              [2,'0.3','shake',False],
              [3,'0.3','shake',False],
              [4,'0.3','milk',False]
             ]

row_df=pd.DataFrame(initial_data,columns = column_names)
item_char = pd.concat([row_df, item_char])
item_char=item_char.set_index('item_ID')
item_char

,sweetness,category,cold_only
item_ID,,,
0,0.9,shake,True
1,0.3,shake,False
2,0.3,shake,False
3,0.3,shake,False
4,0.3,milk,False


In [3]:
#build table: user vs feature, from questionare, how care is each user on each feature, work as weight in calculating similarity.

#1/variance

column_names = ["User_ID", "sweetness", "category",'cold_only']
user_caring = pd.DataFrame(columns = column_names)

temp_1=[  [0,0.8,0.8,0.4],
          [1,0.7,0.7,0.2],
          [2,0.3,0.5,0.5]
             ]

table_temp_1=pd.DataFrame(temp_1,columns = column_names)
user_caring = pd.concat([table_temp_1, user_caring])
user_caring=user_caring.set_index('User_ID')
user_caring

,sweetness,category,cold_only
User_ID,,,
0,0.8,0.8,0.4
1,0.7,0.7,0.2
2,0.3,0.5,0.5


In [4]:
#calculare item similarity based on content
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.preprocessing import LabelEncoder

#LabelEncoder
def labelencoder_dataframe(df):
    for col in df.columns:
        if len(set(df[col]))==2:
            lbl = LabelEncoder()
            lbl.fit(list(df[col].values.astype('str')))
            df[col] = lbl.transform(list(df[col].values.astype('str')))
    return df

item_char=labelencoder_dataframe(item_char)

#Get dummies for catigory variables.
item_char_dum=pd.get_dummies(item_char)

similarity_mat=cosine_similarity(item_char_dum)
# similarity_mat=pd.DataFrame(similarity_mat,columns=item_char.index)
# similarity_mat=similarity_mat.set_index(item_char.index)
similarity_mat


array([[1.        , 0.57735027, 0.57735027, 0.57735027, 0.        ],
       [0.57735027, 1.        , 1.        , 1.        , 0.        ],
       [0.57735027, 1.        , 1.        , 1.        , 0.        ],
       [0.57735027, 1.        , 1.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [5]:
#calculate similarity through image

In [16]:
drive.mount('/content/drive')
path_list=[]
for dirname, _, filenames in os.walk('/content/drive/My Drive/photo'):
    for filename in filenames:
        path_list.append(os.path.join(dirname, filename))
path_list

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['/content/drive/My Drive/photo/0.png',
 '/content/drive/My Drive/photo/1.png',
 '/content/drive/My Drive/photo/2.png',
 '/content/drive/My Drive/photo/3.png',
 '/content/drive/My Drive/photo/4.png']

In [7]:
#import image to a dataframe
img_list=[]
min_width = 800
min_height = 800
for path in path_list:
  temp_1=cv2.imread(path)
  temp_2=cv2.resize(temp_1, (min_width,min_height), interpolation = cv2.INTER_AREA).flatten()
  img_list.append(temp_2)
img_list
img=np.vstack(img_list)
feat_cols = ['pixel'+str(i) for i in range(img.shape[1])]
img_df = pd.DataFrame(img,columns=feat_cols)
#img_df.head()

In [8]:
#PCA to rule out uneccessary features
pca_fresh = PCA(n_components = 5 )
principalComponents_fresh = pca_fresh.fit_transform(img_df.iloc[:,:-1])

# Convert principal components to pandas dataframe 
item_char_pca = pd.DataFrame(data = principalComponents_fresh
             , columns = ['PC1', 'PC2','PC3','PC4','PC5'])
#item_char_pca.head()

In [9]:
#calculare item similarity based on image; no need to creat dummy
similarity_mat_img=cosine_similarity(item_char_pca)
# similarity_mat=pd.DataFrame(similarity_mat,columns=item_char.index)
# similarity_mat=similarity_mat.set_index(item_char.index)
similarity_mat_img

array([[ 1.        , -0.30006396, -0.15126627, -0.17067378, -0.31234554],
       [-0.30006396,  1.        , -0.38808126,  0.11162037, -0.40448883],
       [-0.15126627, -0.38808126,  1.        , -0.35661709, -0.06439105],
       [-0.17067378,  0.11162037, -0.35661709,  1.        , -0.43562033],
       [-0.31234554, -0.40448883, -0.06439105, -0.43562033,  1.        ]])

In [10]:
#build table: user vs item
temp_user_item=[[0,1,1,0,0],
                [0,1,1,2,0],
                [0,1,1,0,3]
                ]

user_item_buy=pd.DataFrame(temp_user_item,columns=list(map(lambda x: 'item'+str(x),item_char.index)))
user_item_buy.index.name='user_id'
user_item_buy

,item0,item1,item2,item3,item4
user_id,,,,,
0,0,1,1,0,0
1,0,1,1,2,0
2,0,1,1,0,3


For a specific user, his/her preference on the $item_j$ is:

preference on $item_j$=similarity $(item_1,item_j)$* number of $item_1$ purchased $+\cdots+$ similarity $(item_n,item_j)$* number of $item_n$ purchased

In [15]:
#recommendation model:

user_id=0
ID_have=np.array(user_item_buy.iloc[user_id])

ID_recmd_sor=np.matmul(similarity_mat, ID_have)

ID_recmd_sor=pd.DataFrame(ID_recmd_sor,columns=['recm_score'])
ID_recmd_sor['recm_score']=preprocessing.minmax_scale(ID_recmd_sor['recm_score'])
ID_recmd_sor.index.name='item_id'
print(ID_recmd_sor)
ID_recmd=ID_recmd_sor.sort_values(by='recm_score',ascending=False).index
ID_recmd=list(ID_recmd)
for id_temp in item_char.index[ID_have>0]:
  ID_recmd.remove(id_temp)



print('The recommend items are'+str(ID_recmd[:3]))#recommend three

         recm_score
item_id            
0           0.57735
1           1.00000
2           1.00000
3           1.00000
4           0.00000
The recommend items are[3, 0, 4]


In [12]:
#cold start

In [13]:
#evaluation